In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [ ]:
cpp_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/cpp_df.csv'
ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ptio2_df.csv'
filtered_ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ptio2_df_filtered.csv'
temperature_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/temperature_df.csv'
lpr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/lpr_df.csv'
hr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/hr_df.csv'
etco2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/etco2_df.csv'
ci_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/ci_df.csv'
prx_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_extracted_data/prx_df.csv'
drug_administration_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/drug_administrations.xlsx'
registry_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/moberg_registry_kssg_post_hoc_modified.xlsx'
paco2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/pdms_data/joined_aBGA.csv'
mainstream_etco2_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/neurocrit_fever/data/PDMS_data/joined_etCO2.csv'
icp_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/neurocrit_fever/data/moberg_extracted_data/icp_df.csv'

In [ ]:
exclude_short_infusions = True
use_filtered_ptio2 = True

In [ ]:
cpp_df = pd.read_csv(cpp_path)
if use_filtered_ptio2:
    ptio2_df = pd.read_csv(filtered_ptio2_path)
else:
    ptio2_df = pd.read_csv(ptio2_path)
temperature_df = pd.read_csv(temperature_path)
lpr_df = pd.read_csv(lpr_path)
hr_df = pd.read_csv(hr_path)
# No associated CI measurements - so code needs no to be executed
# ci_df = pd.read_csv(ci_path)
etco2_df = pd.read_csv(etco2_path)
prx_df = pd.read_csv(prx_path)
icp_df = pd.read_csv(icp_path)
drug_administration_df = pd.read_excel(drug_administration_path)

load pdms data

In [ ]:
registry_df = pd.read_excel(registry_path)
paco2_df = pd.read_csv(paco2_path, sep=';')
mainstream_etco2_df = pd.read_csv(mainstream_etco2_path, sep=';')

In [ ]:
paco2_df = pd.merge(paco2_df, registry_df[['manual_mrn', 'Pat. Nr.']], left_on='FallNr', right_on='manual_mrn', how='left')
paco2_df.drop(columns=['manual_mrn'], inplace=True)
paco2_df.rename(columns={'Pat. Nr.':'pat_nr'}, inplace=True)
paco2_df.rename(columns={'Zeitpunkt_aBGA':'datetime'}, inplace=True)
paco2_df['pCO2_mmHg'] = paco2_df['pCO2'] * 7.50062

In [ ]:
mainstream_etco2_df = pd.merge(mainstream_etco2_df, registry_df[['manual_mrn', 'Pat. Nr.']], left_on='FallNr', right_on='manual_mrn', how='left')
mainstream_etco2_df.drop(columns=['manual_mrn'], inplace=True)
mainstream_etco2_df.rename(columns={'Pat. Nr.':'pat_nr'}, inplace=True)
mainstream_etco2_df.rename(columns={'Zeitpunkt_etCO2':'datetime'}, inplace=True)

# Data Exploration

In [ ]:
drug_administration_df = drug_administration_df[drug_administration_df.monitored]
# exclude if further_exclusion_criterium is not Nan
drug_administration_df = drug_administration_df[pd.isna(drug_administration_df['further_exclusion_criterium'])]

In [ ]:
if exclude_short_infusions:
    drug_administration_df['infusion_duration'] = (pd.to_datetime(drug_administration_df['drug_end']) - pd.to_datetime(drug_administration_df['drug_start'])).dt.total_seconds() / 3600
    print(f'Excluding {drug_administration_df[drug_administration_df["infusion_duration"] <= 1].shape[0]} infusions with duration <= 1h')
    drug_administration_df = drug_administration_df[drug_administration_df['infusion_duration'] > 1]

In [ ]:
drug_administration_df.pat_nr.nunique(), drug_administration_df.shape

In [ ]:
for var_df in [ptio2_df, cpp_df, temperature_df, lpr_df, hr_df, etco2_df, paco2_df, mainstream_etco2_df, prx_df, icp_df]:
    var_df['datetime'] = pd.to_datetime(var_df['datetime'])


In [ ]:
# for every drug administration extract data from -xh to +xh around start
time_window = 12

associated_ptio2_df = pd.DataFrame()
associated_cpp_df = pd.DataFrame()
associated_temperature_df = pd.DataFrame()
associated_hr_df = pd.DataFrame()
associated_lpr_df = pd.DataFrame()
associated_ci_df = pd.DataFrame()
associated_prx_df = pd.DataFrame()
associated_etco2_df = pd.DataFrame()
associated_paco2_df = pd.DataFrame()
associated_mainstream_etco2_df = pd.DataFrame()
associated_icp_df = pd.DataFrame()

for index, row in drug_administration_df.iterrows():
    lower_bound = row['drug_start'] - pd.to_timedelta(time_window, unit='h')
    upper_bound = row['drug_start'] + pd.to_timedelta(time_window, unit='h')
    instance_associated_ptio2_df = ptio2_df[(ptio2_df['pat_nr'] == row['pat_nr'])
                                            & (ptio2_df['datetime'] >= lower_bound) 
                                            & (ptio2_df['datetime'] <= upper_bound)]
    instance_associated_ptio2_df['drug_start'] = row['drug_start']
    instance_associated_ptio2_df['relative_datetime'] = (instance_associated_ptio2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_ptio2_df = pd.concat([associated_ptio2_df, instance_associated_ptio2_df])

    instance_associated_cpp_df = cpp_df[(cpp_df['pat_nr'] == row['pat_nr'])
                                        & (cpp_df['datetime'] >= lower_bound) 
                                        & (cpp_df['datetime'] <= upper_bound)]  
    instance_associated_cpp_df['drug_start'] = row['drug_start']
    instance_associated_cpp_df['relative_datetime'] = (instance_associated_cpp_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_cpp_df = pd.concat([associated_cpp_df, instance_associated_cpp_df])

    instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                        & (temperature_df['datetime'] >= lower_bound)
                                        & (temperature_df['datetime'] <= upper_bound)]
    instance_associated_temperature_df['drug_start'] = row['drug_start']
    instance_associated_temperature_df['relative_datetime'] = (instance_associated_temperature_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_temperature_df = pd.concat([associated_temperature_df, instance_associated_temperature_df])

    instance_associated_hr_df = hr_df[(hr_df['pat_nr'] == row['pat_nr'])
                                        & (hr_df['datetime'] >= lower_bound)   
                                        & (hr_df['datetime'] <= upper_bound)]
    instance_associated_hr_df['drug_start'] = row['drug_start']
    instance_associated_hr_df['relative_datetime'] = (instance_associated_hr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_hr_df = pd.concat([associated_hr_df, instance_associated_hr_df])

    instance_associated_lpr_df = lpr_df[(lpr_df['pat_nr'] == row['pat_nr'])
                                        & (lpr_df['datetime'] >= lower_bound)
                                        & (lpr_df['datetime'] <= upper_bound)]
    instance_associated_lpr_df['drug_start'] = row['drug_start']
    instance_associated_lpr_df['relative_datetime'] = (instance_associated_lpr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_lpr_df = pd.concat([associated_lpr_df, instance_associated_lpr_df])
    
    instance_associated_etco2_df = etco2_df[(etco2_df['pat_nr'] == row['pat_nr'])
                                        & (etco2_df['datetime'] >= lower_bound)
                                        & (etco2_df['datetime'] <= upper_bound)]
    instance_associated_etco2_df['drug_start'] = row['drug_start']
    instance_associated_etco2_df['relative_datetime'] = (instance_associated_etco2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_etco2_df = pd.concat([associated_etco2_df, instance_associated_etco2_df])
    
    instance_associated_paco2_df = paco2_df[(paco2_df['pat_nr'] == row['pat_nr'])
                                        & (paco2_df['datetime'] >= lower_bound)
                                        & (paco2_df['datetime'] <= upper_bound)]
    instance_associated_paco2_df['drug_start'] = row['drug_start']
    instance_associated_paco2_df['relative_datetime'] = (instance_associated_paco2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_paco2_df = pd.concat([associated_paco2_df, instance_associated_paco2_df])
    
    instance_associated_mainstream_etco2_df = mainstream_etco2_df[(mainstream_etco2_df['pat_nr'] == row['pat_nr'])
                                        & (mainstream_etco2_df['datetime'] >= lower_bound)
                                        & (mainstream_etco2_df['datetime'] <= upper_bound)]
    instance_associated_mainstream_etco2_df['drug_start'] = row['drug_start']
    instance_associated_mainstream_etco2_df['relative_datetime'] = (instance_associated_mainstream_etco2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_mainstream_etco2_df = pd.concat([associated_mainstream_etco2_df, instance_associated_mainstream_etco2_df])
    
    instance_associated_prx_df = prx_df[(prx_df['pat_nr'] == row['pat_nr'])
                                        & (prx_df['datetime'] >= lower_bound)
                                        & (prx_df['datetime'] <= upper_bound)]
    instance_associated_prx_df['drug_start'] = row['drug_start']
    instance_associated_prx_df['relative_datetime'] = (instance_associated_prx_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_prx_df = pd.concat([associated_prx_df, instance_associated_prx_df])

    instance_associated_icp_df = icp_df[(icp_df['pat_nr'] == row['pat_nr'])
                                        & (icp_df['datetime'] >= lower_bound)
                                        & (icp_df['datetime'] <= upper_bound)]
    instance_associated_icp_df['drug_start'] = row['drug_start']
    instance_associated_icp_df['relative_datetime'] = (instance_associated_icp_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_icp_df = pd.concat([associated_icp_df, instance_associated_icp_df])
    
    # No associated CI measurements - so code needs no to be executed
    # instance_associated_ci_df = ci_df[(ci_df['pat_nr'] == row['pat_nr'])
    #                                     & (ci_df['datetime'] >= lower_bound)
    #                                     & (ci_df['datetime'] <= upper_bound)]
    # instance_associated_ci_df['drug_start'] = row['drug_start']
    # instance_associated_ci_df['relative_datetime'] = (instance_associated_ci_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    # associated_ci_df = pd.concat([associated_ci_df, instance_associated_ci_df])

    

# Data Cleaning

In [ ]:
allowed_ptio2_range = [0, 200]
allowed_cpp_range = [0, 200]
allowed_temperature_range = [30, 45]
allowed_hr_range = [0, 300]
allowed_lpr_range = [0, 100]
allowed_etco2_range = [0, 100]
allowed_paco2_range = [0.5, 20] # in kpa
allowed_icp_range = [-5, 100]

In [ ]:
# drop values outside of allowed range
associated_ptio2_df = associated_ptio2_df[(associated_ptio2_df['ptio2'] >= allowed_ptio2_range[0]) & (associated_ptio2_df['ptio2'] <= allowed_ptio2_range[1])]
associated_cpp_df = associated_cpp_df[(associated_cpp_df['cpp'] >= allowed_cpp_range[0]) & (associated_cpp_df['cpp'] <= allowed_cpp_range[1])]
associated_temperature_df = associated_temperature_df[(associated_temperature_df['temperature'] >= allowed_temperature_range[0]) & (associated_temperature_df['temperature'] <= allowed_temperature_range[1])]
associated_hr_df = associated_hr_df[(associated_hr_df['hr'] >= allowed_hr_range[0]) & (associated_hr_df['hr'] <= allowed_hr_range[1])]
associated_lpr_df = associated_lpr_df[(associated_lpr_df['lpr'] >= allowed_lpr_range[0]) & (associated_lpr_df['lpr'] <= allowed_lpr_range[1])]
associated_etco2_df = associated_etco2_df[(associated_etco2_df['etco2'] >= allowed_etco2_range[0]) & (associated_etco2_df['etco2'] <= allowed_etco2_range[1])]
associated_paco2_df = associated_paco2_df[(associated_paco2_df['pCO2'] >= allowed_paco2_range[0]) & (associated_paco2_df['pCO2'] <= allowed_paco2_range[1])]
associated_mainstream_etco2_df = associated_mainstream_etco2_df[(associated_mainstream_etco2_df['etCO2'] >= allowed_etco2_range[0]) & (associated_mainstream_etco2_df['etCO2'] <= allowed_etco2_range[1])]
associated_icp_df = associated_icp_df[(associated_icp_df['icp'] >= allowed_icp_range[0])
                                        & (associated_icp_df['icp'] <= allowed_icp_range[1])]

# Data Plotting

In [ ]:
associated_ptio2_df['rounded_relative_datetime'] = associated_ptio2_df['relative_datetime'].round(1)
associated_cpp_df['rounded_relative_datetime'] = associated_cpp_df['relative_datetime'].round(1)
associated_temperature_df['rounded_relative_datetime'] = associated_temperature_df['relative_datetime'].round(1)
associated_hr_df['rounded_relative_datetime'] = associated_hr_df['relative_datetime'].round(1)
associated_lpr_df['rounded_relative_datetime'] = associated_lpr_df['relative_datetime'].round(1)
associated_etco2_df['rounded_relative_datetime'] = associated_etco2_df['relative_datetime'].round(1)
associated_paco2_df['rounded_relative_datetime'] = associated_paco2_df['relative_datetime'].round(1)
associated_mainstream_etco2_df['rounded_relative_datetime'] = associated_mainstream_etco2_df['relative_datetime'].round(1)
associated_prx_df['rounded_relative_datetime'] = associated_prx_df['relative_datetime'].round(1)
associated_icp_df['rounded_relative_datetime'] = associated_icp_df['relative_datetime'].round(1)

# Full figure

In [ ]:
# plot a line plot of ptio2 and cpp values over relative time
plot_temperature = True
plot_temperature_left = True
plot_hr = True
plot_lpr = False
plot_etco2 = False
plot_paco2 = False
plot_prx = True

plot_diclofenac_infusion = True

n_in_label = False
broken_y_axis = True
label_fontsize = 18
legend_fontsize = 16

secondary_axis_alpha = 0.8

if broken_y_axis:
    fig, (ax, ax0) = plt.subplots(2, 1, sharex=True, figsize=(20, 10), height_ratios=[0.95, 0.05])
    fig.subplots_adjust(hspace=0.05)  # adjust space between Axes
    
    ax0.set_ylim(0, 1.)  # outliers only
    ax.set_ylim(20, 36)  # most of the data
    
    # hide the spines between ax and ax2
    ax.spines.bottom.set_visible(False)
    ax0.spines.top.set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax0.xaxis.tick_bottom()
    
    d = .5  # proportion of vertical to horizontal extent of the slanted line
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                  linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax.plot([0, 1], [0, 0], transform=ax.transAxes, **kwargs)
    ax0.plot([0, 1], [1, 1], transform=ax0.transAxes, **kwargs)
    
    # remove y tick from ax0
    ax0.set_yticks([0])
else:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

pto2_label = 'PtO2'
if n_in_label:
    pto2_label = f'PtO2 (n={associated_ptio2_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=associated_ptio2_df, ax=ax, 
             label=pto2_label, legend=False, linewidth=2)
# create second y axis for cpp values
ax2 = ax.twinx()
cpp_label = 'CPP'
if n_in_label:
    cpp_label = f'CPP (n={associated_cpp_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=associated_cpp_df, ax=ax2, color='magenta', 
             label=cpp_label, legend=False, alpha=secondary_axis_alpha)

if plot_temperature:
    ax3 = ax.twinx()
    temp_label = 'Temperature'
    if n_in_label:
        temp_label = f'Temperature (n={associated_temperature_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=associated_temperature_df, ax=ax3, color='green', 
                 label=temp_label, legend=False, alpha=secondary_axis_alpha)
    ax3.set_ylabel('Temperature (°C)', fontsize=label_fontsize)
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))
    
if plot_hr:
    ax4 = ax.twinx()
    hr_label = 'Heart rate'
    if n_in_label:
        hr_label = f'Heart rate (n={associated_hr_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=associated_hr_df, ax=ax4, color='#7b002c', 
                 label=hr_label, legend=False, alpha=secondary_axis_alpha)
    ax4.set_ylabel('Heart rate (bpm)', fontsize=label_fontsize)
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120)) 

if plot_lpr:
    ax5 = ax.twinx()
    lpr_label = 'LPR'
    if n_in_label:
        lpr_label = f'LPR (n={associated_lpr_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=associated_lpr_df.reset_index(), ax=ax5, color='orange', 
                 label=lpr_label, legend=False, alpha=secondary_axis_alpha)
    ax5.set_ylabel('LPR', fontsize=label_fontsize)
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))

if plot_etco2:
    ax6 = ax.twinx()
    etco2_label = 'EtCO2'
    if n_in_label:
        etco2_label = f'EtCO2 (n={associated_etco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='etco2', data=associated_etco2_df.reset_index(), ax=ax6, color='orange', 
                 label=etco2_label, legend=False, alpha=secondary_axis_alpha)
    ax6.set_ylabel('EtCO2 (mmHg)', fontsize=label_fontsize)
    ax6.grid(None)
    ax6.spines['right'].set_position(('outward', 180))
    
if plot_paco2:
    ax7 = ax.twinx()
    pac02_label = 'PaCO2'
    if n_in_label:
        pac02_label = f'PaCO2 (n={associated_paco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='pCO2_mmHg', data=associated_paco2_df.reset_index(), ax=ax7, color='orange',
                    label=pac02_label, legend=False, alpha=secondary_axis_alpha)
    ax7.set_ylabel('PaCO2 (mmHg)', fontsize=label_fontsize)
    ax7.grid(None)
    ax7.spines['right'].set_position(('outward', 180))
    
if broken_y_axis:
    ax0.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
else:
    ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtO2 (mmHg)', fontsize=label_fontsize)
ax2.set_ylabel('CPP (mmHg)', fontsize=label_fontsize)

ax2.set_ylim(60,100)
ax2.grid(None)

# show legend
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3
    
if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4
    
if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5

if plot_etco2:
    lines6, labels6 = ax6.get_legend_handles_labels()
    legend_lines += lines6
    legend_labels += labels6
    
if plot_paco2:
    lines7, labels7 = ax7.get_legend_handles_labels()
    legend_lines += lines7
    legend_labels += labels7

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

# # add vertical line at x=0 with text label with "Diclofenac administration"
# ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
# ax.text(0.5, 0.12, 'Start of infusion', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

if plot_diclofenac_infusion:
    if broken_y_axis:
        ax0.plot([0, 12], [0.3, 0.3], color='grey', linewidth=10, alpha=0.5)
        ax0.text(6, 0.85, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)
    else:
        ax.plot([0, 12], [20.5, 20.5], color='grey', linewidth=10, alpha=0.5)
        ax.text(6, 21, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)
        
# switch y axes of temperature (ax3) and PtO2 (ax)
ax3.yaxis.set_label_position("left")
ax3.yaxis.tick_left()

ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
# plot a line plot of ptio2 and cpp values over relative time
plot_temperature = True
plot_hr = True
plot_lpr = False
plot_etco2 = False
plot_paco2 = False
plot_prx = True

plot_diclofenac_infusion = True

n_in_label = False
broken_y_axis = True
label_fontsize = 18
legend_fontsize = 16

secondary_axis_alpha = 0.8

if broken_y_axis:
    fig, (ax, ax0) = plt.subplots(2, 1, sharex=True, figsize=(20, 10), height_ratios=[0.95, 0.05])
    fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

    ax0.set_ylim(0, 1.)  # outliers only
    ax.set_ylim(20, 36)  # most of the data

    # hide the spines between ax and ax2
    ax.spines.bottom.set_visible(False)
    ax0.spines.top.set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax0.xaxis.tick_bottom()

    d = .5  # proportion of vertical to horizontal extent of the slanted line
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                  linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax.plot([0, 1], [0, 0], transform=ax.transAxes, **kwargs)
    ax0.plot([0, 1], [1, 1], transform=ax0.transAxes, **kwargs)

    # remove y tick from ax0
    ax0.set_yticks([0])
else:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

pto2_label = 'PtO2'
if n_in_label:
    pto2_label = f'PtO2 (n={associated_ptio2_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=associated_ptio2_df, ax=ax,
             label=pto2_label, legend=False, linewidth=2)
# create second y axis for cpp values
ax2 = ax.twinx()
cpp_label = 'CPP'
if n_in_label:
    cpp_label = f'CPP (n={associated_cpp_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=associated_cpp_df, ax=ax2, color='magenta',
             label=cpp_label, legend=False, alpha=secondary_axis_alpha)

if plot_temperature:
    ax3 = ax.twinx()
    temp_label = 'Temperature'
    if n_in_label:
        temp_label = f'Temperature (n={associated_temperature_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=associated_temperature_df, ax=ax3, color='green',
                 label=temp_label, legend=False, alpha=secondary_axis_alpha)
    ax3.set_ylabel('Temperature (°C)', fontsize=label_fontsize)
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))

if plot_hr:
    ax4 = ax.twinx()
    hr_label = 'Heart rate'
    if n_in_label:
        hr_label = f'Heart rate (n={associated_hr_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=associated_hr_df, ax=ax4, color='#7b002c',
                 label=hr_label, legend=False, alpha=secondary_axis_alpha)
    ax4.set_ylabel('Heart rate (bpm)', fontsize=label_fontsize)
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120))

if plot_lpr:
    ax5 = ax.twinx()
    lpr_label = 'LPR'
    if n_in_label:
        lpr_label = f'LPR (n={associated_lpr_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=associated_lpr_df.reset_index(), ax=ax5, color='orange',
                 label=lpr_label, legend=False, alpha=secondary_axis_alpha)
    ax5.set_ylabel('LPR', fontsize=label_fontsize)
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))

if plot_etco2:
    ax6 = ax.twinx()
    etco2_label = 'EtCO2'
    if n_in_label:
        etco2_label = f'EtCO2 (n={associated_etco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='etco2', data=associated_etco2_df.reset_index(), ax=ax6,
                 color='orange',
                 label=etco2_label, legend=False, alpha=secondary_axis_alpha)
    ax6.set_ylabel('EtCO2 (mmHg)', fontsize=label_fontsize)
    ax6.grid(None)
    ax6.spines['right'].set_position(('outward', 180))

if plot_paco2:
    ax7 = ax.twinx()
    pac02_label = 'PaCO2'
    if n_in_label:
        pac02_label = f'PaCO2 (n={associated_paco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='pCO2_mmHg', data=associated_paco2_df.reset_index(), ax=ax7,
                 color='orange',
                 label=pac02_label, legend=False, alpha=secondary_axis_alpha)
    ax7.set_ylabel('PaCO2 (mmHg)', fontsize=label_fontsize)
    ax7.grid(None)
    ax7.spines['right'].set_position(('outward', 180))

if broken_y_axis:
    ax0.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
else:
    ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtO2 (mmHg)', fontsize=label_fontsize)
ax2.set_ylabel('CPP (mmHg)', fontsize=label_fontsize)

ax2.set_ylim(60, 100)
ax2.grid(None)

# show legend
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3

if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4

if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5

if plot_etco2:
    lines6, labels6 = ax6.get_legend_handles_labels()
    legend_lines += lines6
    legend_labels += labels6

if plot_paco2:
    lines7, labels7 = ax7.get_legend_handles_labels()
    legend_lines += lines7
    legend_labels += labels7

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

# # add vertical line at x=0 with text label with "Diclofenac administration"
# ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
# ax.text(0.5, 0.12, 'Start of infusion', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

if plot_diclofenac_infusion:
    if broken_y_axis:
        ax0.plot([0, 12], [0.3, 0.3], color='grey', linewidth=10, alpha=0.5)
        ax0.text(6, 0.85, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center',
                 fontsize=legend_fontsize)
    else:
        ax.plot([0, 12], [20.5, 20.5], color='grey', linewidth=10, alpha=0.5)
        ax.text(6, 21, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center',
                fontsize=legend_fontsize)

# switch y axes of temperature (ax3) and PtO2 (ax)
ax3.yaxis.set_label_position("left")
ax3.yaxis.tick_left()

ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()

# plot a line plot of ptio2 and cpp values over relative time
plot_temperature = True
plot_hr = True
plot_lpr = False
plot_etco2 = False
plot_paco2 = False
plot_prx = True

plot_diclofenac_infusion = True

n_in_label = False
broken_y_axis = True
label_fontsize = 18
legend_fontsize = 16

secondary_axis_alpha = 0.8

if broken_y_axis:
    fig, (ax, ax0) = plt.subplots(2, 1, sharex=True, figsize=(20, 10), height_ratios=[0.95, 0.05])
    fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

    ax0.set_ylim(0, 1.)  # outliers only
    ax.set_ylim(20, 36)  # most of the data

    # hide the spines between ax and ax2
    ax.spines.bottom.set_visible(False)
    ax0.spines.top.set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax0.xaxis.tick_bottom()

    d = .5  # proportion of vertical to horizontal extent of the slanted line
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                  linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax.plot([0, 1], [0, 0], transform=ax.transAxes, **kwargs)
    ax0.plot([0, 1], [1, 1], transform=ax0.transAxes, **kwargs)

    # remove y tick from ax0
    ax0.set_yticks([0])
else:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

pto2_label = 'PtO2'
if n_in_label:
    pto2_label = f'PtO2 (n={associated_ptio2_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=associated_ptio2_df, ax=ax,
             label=pto2_label, legend=False, linewidth=2)
# create second y axis for cpp values
ax2 = ax.twinx()
cpp_label = 'CPP'
if n_in_label:
    cpp_label = f'CPP (n={associated_cpp_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=associated_cpp_df, ax=ax2, color='magenta',
             label=cpp_label, legend=False, alpha=secondary_axis_alpha)

if plot_temperature:
    ax3 = ax.twinx()
    temp_label = 'Temperature'
    if n_in_label:
        temp_label = f'Temperature (n={associated_temperature_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=associated_temperature_df, ax=ax3, color='green',
                 label=temp_label, legend=False, alpha=secondary_axis_alpha)
    ax3.set_ylabel('Temperature (°C)', fontsize=label_fontsize)
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))

if plot_hr:
    ax4 = ax.twinx()
    hr_label = 'Heart rate'
    if n_in_label:
        hr_label = f'Heart rate (n={associated_hr_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=associated_hr_df, ax=ax4, color='#7b002c',
                 label=hr_label, legend=False, alpha=secondary_axis_alpha)
    ax4.set_ylabel('Heart rate (bpm)', fontsize=label_fontsize)
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120))

if plot_lpr:
    ax5 = ax.twinx()
    lpr_label = 'LPR'
    if n_in_label:
        lpr_label = f'LPR (n={associated_lpr_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=associated_lpr_df.reset_index(), ax=ax5, color='orange',
                 label=lpr_label, legend=False, alpha=secondary_axis_alpha)
    ax5.set_ylabel('LPR', fontsize=label_fontsize)
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))

if plot_etco2:
    ax6 = ax.twinx()
    etco2_label = 'EtCO2'
    if n_in_label:
        etco2_label = f'EtCO2 (n={associated_etco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='etco2', data=associated_etco2_df.reset_index(), ax=ax6, color='orange',
                 label=etco2_label, legend=False, alpha=secondary_axis_alpha)
    ax6.set_ylabel('EtCO2 (mmHg)', fontsize=label_fontsize)
    ax6.grid(None)
    ax6.spines['right'].set_position(('outward', 180))

if plot_paco2:
    ax7 = ax.twinx()
    pac02_label = 'PaCO2'
    if n_in_label:
        pac02_label = f'PaCO2 (n={associated_paco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='pCO2_mmHg', data=associated_paco2_df.reset_index(), ax=ax7, color='orange',
                 label=pac02_label, legend=False, alpha=secondary_axis_alpha)
    ax7.set_ylabel('PaCO2 (mmHg)', fontsize=label_fontsize)
    ax7.grid(None)
    ax7.spines['right'].set_position(('outward', 180))

if broken_y_axis:
    ax0.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
else:
    ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtO2 (mmHg)', fontsize=label_fontsize)
ax2.set_ylabel('CPP (mmHg)', fontsize=label_fontsize)

ax2.set_ylim(60,100)
ax2.grid(None)

# show legend
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3

if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4

if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5

if plot_etco2:
    lines6, labels6 = ax6.get_legend_handles_labels()
    legend_lines += lines6
    legend_labels += labels6

if plot_paco2:
    lines7, labels7 = ax7.get_legend_handles_labels()
    legend_lines += lines7
    legend_labels += labels7

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

# # add vertical line at x=0 with text label with "Diclofenac administration"
# ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
# ax.text(0.5, 0.12, 'Start of infusion', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

if plot_diclofenac_infusion:
    if broken_y_axis:
        ax0.plot([0, 12], [0.3, 0.3], color='grey', linewidth=10, alpha=0.5)
        ax0.text(6, 0.85, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)
    else:
        ax.plot([0, 12], [20.5, 20.5], color='grey', linewidth=10, alpha=0.5)
        ax.text(6, 21, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)

if plot_temperature_left:
    # switch y axes of temperature (ax3) and PtO2 (ax)
    ax3.yaxis.set_label_position("left")
    ax3.yaxis.tick_left()
    
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax2.spines['right'].set_position(('outward', 60))


In [ ]:
# fig.savefig('/Users/jk1/Downloads/temp_and_ptio2.png', dpi=600, bbox_inches='tight')

## Plot PRx with CPP and PtiO2

In [ ]:
sns.lineplot(x='rounded_relative_datetime', y='prx', data=associated_prx_df, color='lightslategrey', 
                 label='prx_label', legend=False, alpha=secondary_axis_alpha)

In [ ]:
# plot a line plot of ptio2 and cpp values over relative time
plot_cpp = False
plot_temperature = False
plot_prx = True

plot_diclofenac_infusion = False

n_in_label = False
broken_y_axis = True
label_fontsize = 18
legend_fontsize = 16

secondary_axis_alpha = 1

prx_patients = associated_prx_df.pat_nr.unique()
prx_associated_ptio2_df = associated_ptio2_df[associated_ptio2_df.pat_nr.isin(prx_patients)]
prx_associated_cpp_df = associated_cpp_df[associated_cpp_df.pat_nr.isin(prx_patients)]
prx_associated_temperature_df = associated_temperature_df[associated_temperature_df.pat_nr.isin(prx_patients)]

if broken_y_axis:
    fig, (ax, ax0) = plt.subplots(2, 1, sharex=True, figsize=(20, 10), height_ratios=[0.95, 0.05])
    fig.subplots_adjust(hspace=0.05)  # adjust space between Axes
    
    ax0.set_ylim(0, 1.)  # outliers only
    ax.set_ylim(12, 23)  # most of the data
    
    # hide the spines between ax and ax2
    ax.spines.bottom.set_visible(False)
    ax0.spines.top.set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax0.xaxis.tick_bottom()
    
    d = .5  # proportion of vertical to horizontal extent of the slanted line
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                  linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax.plot([0, 1], [0, 0], transform=ax.transAxes, **kwargs)
    ax0.plot([0, 1], [1, 1], transform=ax0.transAxes, **kwargs)
    
    # remove y tick from ax0
    ax0.set_yticks([0])
else:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

pto2_label = 'PtO2'
if n_in_label:
    pto2_label = f'PtO2 (n={prx_associated_ptio2_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=prx_associated_ptio2_df, ax=ax, 
             label=pto2_label, legend=False, linewidth=2)

if plot_cpp:
    # create second y axis for cpp values
    ax2 = ax.twinx()
    cpp_label = 'CPP'
    if n_in_label:
        cpp_label = f'CPP (n={prx_associated_cpp_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='cpp', data=prx_associated_cpp_df, ax=ax2, color='magenta', 
                 label=cpp_label, legend=False, alpha=secondary_axis_alpha)
    ax2.set_ylim(60,100)
    ax2.grid(None)

if plot_temperature:
    ax3 = ax.twinx()
    temp_label = 'Temperature'
    if n_in_label:
        temp_label = f'Temperature (n={prx_associated_temperature_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=prx_associated_temperature_df, ax=ax3, color='green', 
                 label=temp_label, legend=False, alpha=secondary_axis_alpha)
    ax3.set_ylabel('Temperature (°C)', fontsize=label_fontsize)
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))
    
if plot_prx:
    ax4 = ax.twinx()
    prx_label = 'PRx'
    if n_in_label:
        prx_label = f'PRx (n={associated_prx_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='prx', data=associated_prx_df, ax=ax4, color='salmon', 
                 label=prx_label, legend=False, alpha=secondary_axis_alpha)
    ax4.set_ylabel('PRx', fontsize=label_fontsize)
    ax4.grid(None)
    # ax4.spines['right'].set_position(('outward', 120))
    
if broken_y_axis:
    ax0.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
else:
    ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtO2 (mmHg)', fontsize=label_fontsize)


# show legend
legend_lines, legend_labels = ax.get_legend_handles_labels()

if plot_cpp:
    lines2, labels2 = ax2.get_legend_handles_labels()
    legend_lines += lines2
    legend_labels += labels2
    ax2.set_ylabel('CPP (mmHg)', fontsize=label_fontsize)

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3

if plot_prx:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

if plot_diclofenac_infusion:
    if broken_y_axis:
        ax0.plot([0, 12], [0.3, 0.3], color='grey', linewidth=10, alpha=0.5)
        ax0.text(6, 0.85, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)
    else:
        ax.plot([0, 12], [20.5, 20.5], color='grey', linewidth=10, alpha=0.5)
        ax.text(6, 21, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)

In [ ]:
# fig.savefig('/Users/jk1/Downloads/prx_subset.png', dpi=600, bbox_inches='tight')

## LPR and PtO2

In [ ]:
# plot a line plot of ptio2 and cpp values over relative time
plot_cpp = False
plot_lpr = True

plot_diclofenac_infusion = False

n_in_label = False
broken_y_axis = True
label_fontsize = 18
legend_fontsize = 16
secondary_axis_alpha = 1

lpr_patients = associated_lpr_df.pat_nr.unique()
associated_lpr_df['rounded_relative_datetime'] = associated_lpr_df['relative_datetime'].round()
lpr_associated_ptio2_df = associated_ptio2_df[associated_ptio2_df.pat_nr.isin(lpr_patients)]
lpr_associated_ptio2_df['rounded_relative_datetime'] = (lpr_associated_ptio2_df['relative_datetime'] * 4).round() / 4
lpr_associated_cpp_df = associated_cpp_df[associated_cpp_df.pat_nr.isin(lpr_patients)]

if broken_y_axis:
    fig, (ax, ax0) = plt.subplots(2, 1, sharex=True, figsize=(20, 10), height_ratios=[0.95, 0.05])
    fig.subplots_adjust(hspace=0.05)  # adjust space between Axes
    
    ax0.set_ylim(0, 1.)  # outliers only
    ax.set_ylim(18, 36)  # most of the data
    
    # hide the spines between ax and ax2
    ax.spines.bottom.set_visible(False)
    ax0.spines.top.set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax0.xaxis.tick_bottom()
    
    d = .5  # proportion of vertical to horizontal extent of the slanted line
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                  linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax.plot([0, 1], [0, 0], transform=ax.transAxes, **kwargs)
    ax0.plot([0, 1], [1, 1], transform=ax0.transAxes, **kwargs)
    
    # remove y tick from ax0
    ax0.set_yticks([0])
else:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

pto2_label = 'PtO2'
if n_in_label:
    pto2_label = f'PtO2 (n={lpr_associated_ptio2_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=lpr_associated_ptio2_df, ax=ax, 
             label=pto2_label, legend=False, linewidth=2)

if plot_cpp:
    # create second y axis for cpp values
    ax2 = ax.twinx()
    cpp_label = 'CPP'
    if n_in_label:
        cpp_label = f'CPP (n={lpr_associated_cpp_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='cpp', data=lpr_associated_cpp_df, ax=ax2, color='magenta', 
                 label=cpp_label, legend=False, alpha=secondary_axis_alpha)
    ax2.set_ylim(60,100)
    ax2.grid(None)

if plot_lpr:
    ax5 = ax.twinx()
    lpr_label = 'LPR'
    if n_in_label:
        lpr_label = f'LPR (n={associated_lpr_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=associated_lpr_df.reset_index(), ax=ax5, color='rosybrown', 
                 label=lpr_label, legend=False, alpha=secondary_axis_alpha)
    ax5.set_ylabel('LPR', fontsize=label_fontsize)
    ax5.grid(None)
    # ax5.spines['right'].set_position(('outward', 180))

if broken_y_axis:
    ax0.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
else:
    ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtO2 (mmHg)', fontsize=label_fontsize)


# show legend
legend_lines, legend_labels = ax.get_legend_handles_labels()

if plot_cpp:
    lines2, labels2 = ax2.get_legend_handles_labels()
    legend_lines += lines2
    legend_labels += labels2
    ax2.set_ylabel('CPP (mmHg)', fontsize=label_fontsize)

if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

# # add vertical line at x=0 with text label with "Diclofenac administration"
# ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
# ax.text(0.5, 0.12, 'Start of infusion', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

if plot_diclofenac_infusion:
    if broken_y_axis:
        ax0.plot([0, 12], [0.3, 0.3], color='grey', linewidth=10, alpha=0.5)
        ax0.text(6, 0.85, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)
    else:
        ax.plot([0, 12], [20.5, 20.5], color='grey', linewidth=10, alpha=0.5)
        ax.text(6, 21, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center', fontsize=legend_fontsize)
        
ax5.set_ylim(10, 47)

In [ ]:
# fig.savefig('/Users/jk1/Downloads/lpr_subset.png', dpi=600, bbox_inches='tight')

## Relation with PaCO2 and EtCO2

In [ ]:
associated_ptio2_df['rounded_relative_datetime'] = associated_ptio2_df['relative_datetime'].round(1)
associated_cpp_df['rounded_relative_datetime'] = associated_cpp_df['relative_datetime'].round(1)
associated_temperature_df['rounded_relative_datetime'] = associated_temperature_df['relative_datetime'].round(1)

associated_paco2_df['rounded_relative_datetime'] = (associated_paco2_df['relative_datetime'] * 2).round() / 2
associated_mainstream_etco2_df['rounded_relative_datetime'] = (associated_mainstream_etco2_df['relative_datetime'] * 2).round() / 2

# plot a line plot of ptio2 and cpp values over relative time
plot_temperature = False
plot_cpp = False
plot_etco2 = True
plot_paco2 = True

plot_diclofenac_infusion = False

n_in_label = False
broken_y_axis = True
label_fontsize = 18
legend_fontsize = 16

secondary_axis_alpha = 0.8

if broken_y_axis:
    fig, (ax, ax0) = plt.subplots(2, 1, sharex=True, figsize=(20, 10), height_ratios=[0.95, 0.05])
    fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

    ax0.set_ylim(0, 1.)  # outliers only
    ax.set_ylim(20, 36)  # most of the data

    # hide the spines between ax and ax2
    ax.spines.bottom.set_visible(False)
    ax0.spines.top.set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax0.xaxis.tick_bottom()

    d = .5  # proportion of vertical to horizontal extent of the slanted line
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                  linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax.plot([0, 1], [0, 0], transform=ax.transAxes, **kwargs)
    ax0.plot([0, 1], [1, 1], transform=ax0.transAxes, **kwargs)

    # remove y tick from ax0
    ax0.set_yticks([0])
else:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

pto2_label = 'PtO2'
if n_in_label:
    pto2_label = f'PtO2 (n={associated_ptio2_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=associated_ptio2_df, ax=ax,
             label=pto2_label, legend=False, linewidth=2)

if plot_cpp:
    # create second y axis for cpp values
    ax2 = ax.twinx()
    cpp_label = 'CPP'
    if n_in_label:
        cpp_label = f'CPP (n={associated_cpp_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='cpp', data=associated_cpp_df, ax=ax2, color='magenta',
                 label=cpp_label, legend=False, alpha=secondary_axis_alpha)
    ax2.set_ylim(60, 100)
    ax2.grid(None)

if plot_diclofenac_infusion:
    if broken_y_axis:
        ax0.plot([0, 12], [0.3, 0.3], color='grey', linewidth=10, alpha=0.5)
        ax0.text(6, 0.85, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center',
                 fontsize=legend_fontsize)
    else:
        ax.plot([0, 12], [20.5, 20.5], color='grey', linewidth=10, alpha=0.5)
        ax.text(6, 21, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center',
                fontsize=legend_fontsize)

if plot_etco2:
    ax6 = ax.twinx()
    etco2_label = 'EtCO2'
    if n_in_label:
        etco2_label = f'EtCO2 (n={associated_mainstream_etco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='etCO2', data=associated_mainstream_etco2_df.reset_index(), ax=ax6,
                 color='orange',
                 label=etco2_label, legend=False, alpha=secondary_axis_alpha)
    ax6.set_ylabel('EtCO2 (mmHg)', fontsize=label_fontsize)
    ax6.grid(None)
    ax6.spines['right'].set_position(('outward', 60))
    ax6.set_ylim(25, 50)

if plot_paco2:
    ax7 = ax.twinx()
    pac02_label = 'PaCO2'
    if n_in_label:
        pac02_label = f'PaCO2 (n={associated_paco2_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='pCO2_mmHg', data=associated_paco2_df.reset_index(), ax=ax7,
                 color='lightcoral',
                 label=pac02_label, legend=False, alpha=secondary_axis_alpha)
    ax7.set_ylabel('PaCO2 (mmHg)', fontsize=label_fontsize)
    ax7.grid(None)
    ax7.set_ylim(25, 50)
    # ax7.spines['right'].set_position(('outward', 180))

if broken_y_axis:
    ax0.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
else:
    ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtO2 (mmHg)', fontsize=label_fontsize)

# show legend
legend_lines, legend_labels = ax.get_legend_handles_labels()

if plot_cpp:
    lines2, labels2 = ax2.get_legend_handles_labels()
    legend_lines += lines2
    legend_labels += labels2
    ax2.set_ylabel('CPP (mmHg)', fontsize=label_fontsize)

if plot_etco2:
    lines6, labels6 = ax6.get_legend_handles_labels()
    legend_lines += lines6
    legend_labels += labels6

if plot_paco2:
    lines7, labels7 = ax7.get_legend_handles_labels()
    legend_lines += lines7
    legend_labels += labels7

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

# # add vertical line at x=0 with text label with "Diclofenac administration"
# ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
# ax.text(0.5, 0.12, 'Start of infusion', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

if plot_diclofenac_infusion:
    if broken_y_axis:
        ax0.plot([0, 12], [0.3, 0.3], color='grey', linewidth=10, alpha=0.5)
        ax0.text(6, 0.85, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center',
                 fontsize=legend_fontsize)
    else:
        ax.plot([0, 12], [20.5, 20.5], color='grey', linewidth=10, alpha=0.5)
        ax.text(6, 21, 'Diclofenac infusion', horizontalalignment='center', verticalalignment='center',
                fontsize=legend_fontsize)

In [ ]:
# fig.savefig('/Users/jk1/Downloads/ptio2_etco2_paco2.png', dpi=600, bbox_inches='tight')

In [ ]:
# plot a line plot of ptio2 and icp values over relative time
plot_icp = True

n_in_label = False
broken_y_axis = True
label_fontsize = 18
legend_fontsize = 16
secondary_axis_alpha = 1

if broken_y_axis:
    fig, (ax, ax0) = plt.subplots(2, 1, sharex=True, figsize=(20, 10), height_ratios=[0.95, 0.05])
    fig.subplots_adjust(hspace=0.05)  # adjust space between Axes

    ax0.set_ylim(0, 1.)  # outliers only
    ax.set_ylim(20, 36)  # most of the data

    # hide the spines between ax and ax2
    ax.spines.bottom.set_visible(False)
    ax0.spines.top.set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop=False)  # don't put tick labels at the top
    ax0.xaxis.tick_bottom()

    d = .5  # proportion of vertical to horizontal extent of the slanted line
    kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
                  linestyle="none", color='k', mec='k', mew=1, clip_on=False)
    ax.plot([0, 1], [0, 0], transform=ax.transAxes, **kwargs)
    ax0.plot([0, 1], [1, 1], transform=ax0.transAxes, **kwargs)

    # remove y tick from ax0
    ax0.set_yticks([0])

else:
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

pto2_label = 'PtO2'
if n_in_label:
    pto2_label = f'PtO2 (n={associated_ptio2_df.pat_nr.nunique()})'
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=associated_ptio2_df, ax=ax,
             label=pto2_label, legend=False, linewidth=2)


if plot_icp:
    # create second y axis for icp values
    ax2 = ax.twinx()
    icp_label = 'ICP'
    if n_in_label:
        cpp_label = f'ICP (n={associated_icp_df.pat_nr.nunique()})'
    sns.lineplot(x='rounded_relative_datetime', y='icp', data=associated_icp_df, ax=ax2, color='magenta',
                 label=icp_label, legend=False, alpha=secondary_axis_alpha)
    ax2.set_ylim(0, 30)
    ax2.grid(None)

if broken_y_axis:
    ax0.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
else:
    ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtO2 (mmHg)', fontsize=label_fontsize)

# show legend
legend_lines, legend_labels = ax.get_legend_handles_labels()

if plot_icp:
    lines2, labels2 = ax2.get_legend_handles_labels()
    legend_lines += lines2
    legend_labels += labels2
    ax2.set_ylabel('ICP (mmHg)', fontsize=label_fontsize)

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

plt.show()

In [ ]:
# fig.savefig('/Users/jk1/Downloads/ptio2_icp.png', dpi=600, bbox_inches='tight')

# Pre / Post comparison

In [ ]:
# boxplot pre / post ptio2
associated_ptio2_df['pre_post'] = 'pre'
associated_ptio2_df.loc[associated_ptio2_df['relative_datetime'] > 0, 'pre_post'] = 'post'
sns.boxplot(x='pre_post', y='ptio2', hue='pre_post', data=associated_ptio2_df)


In [ ]:
# compare pre / post ptio2 with mixed effect model
mixed_model = smf.mixedlm("ptio2 ~ pre_post", associated_ptio2_df, groups=associated_ptio2_df['pat_nr'])

In [ ]:
mdf = mixed_model.fit()
# print formula
print(mdf.model.formula)
print(mdf.summary())

In [ ]:
# compare 1h pre vs 12h post
xh_post = 12
fever_peak_associated_ptio2_df = associated_ptio2_df.copy()
fever_peak_associated_ptio2_df['pre_post'] = np.nan
fever_peak_associated_ptio2_df.loc[(fever_peak_associated_ptio2_df['relative_datetime'] > 0, 'pre_post')] = 'post'

fever_peak_associated_ptio2_df.loc[(fever_peak_associated_ptio2_df['relative_datetime'] > 0)
                                & (fever_peak_associated_ptio2_df['relative_datetime'] < xh_post), 'pre_post'] = 'post'

fever_peak_associated_ptio2_df.loc[(fever_peak_associated_ptio2_df['relative_datetime'] < 0) 
                              & (fever_peak_associated_ptio2_df['relative_datetime'] > -1), 'pre_post'] = 'pre'

In [ ]:
# boxplot
sns.boxplot(x='pre_post', y='ptio2', hue='pre_post', data=fever_peak_associated_ptio2_df.reset_index())

In [ ]:
print(fever_peak_associated_ptio2_df[fever_peak_associated_ptio2_df['pre_post'] == 'pre']['ptio2'].median(), f'(IQR: {fever_peak_associated_ptio2_df[fever_peak_associated_ptio2_df["pre_post"] == "pre"]["ptio2"].quantile(0.25)} - {fever_peak_associated_ptio2_df[fever_peak_associated_ptio2_df["pre_post"] == "pre"]["ptio2"].quantile(0.75)})')
print(fever_peak_associated_ptio2_df[fever_peak_associated_ptio2_df['pre_post'] == 'post']['ptio2'].median(), f'(IQR: {fever_peak_associated_ptio2_df[fever_peak_associated_ptio2_df["pre_post"] == "post"]["ptio2"].quantile(0.25)} - {fever_peak_associated_ptio2_df[fever_peak_associated_ptio2_df["pre_post"] == "post"]["ptio2"].quantile(0.75)})')

In [ ]:
# mixed model
fever_peak_associated_ptio2_df.dropna(subset=['pre_post'], inplace=True)
peak_fever_ptio2_mixed_model = smf.mixedlm("ptio2 ~ pre_post", fever_peak_associated_ptio2_df, groups=fever_peak_associated_ptio2_df['pat_nr'])
mdf = peak_fever_ptio2_mixed_model.fit()
# print formula
print(mdf.model.formula)
print(mdf.summary())

# compare pre / post temperature

In [ ]:
# compare pre / post temperature
associated_temperature_df['pre_post'] = 'pre'
associated_temperature_df.loc[associated_temperature_df['relative_datetime'] > 0, 'pre_post'] = 'post'
sns.boxplot(x='pre_post', y='temperature', hue='pre_post', data=associated_temperature_df)

In [ ]:
associated_temperature_df[associated_temperature_df['pre_post'] == 'pre']['temperature'].median(), associated_temperature_df[associated_temperature_df['pre_post'] == 'post']['temperature'].median()

In [ ]:
# compare 1h pre and Xh post, do not use data before -1h
xh_post = 12
fever_peak_associated_temperature_df = associated_temperature_df.copy()
fever_peak_associated_temperature_df['pre_post'] = np.nan
fever_peak_associated_temperature_df.loc[(fever_peak_associated_temperature_df['relative_datetime'] > 0, 'pre_post')] = 'post'

fever_peak_associated_temperature_df.loc[(fever_peak_associated_temperature_df['relative_datetime'] > 0)
                                & (fever_peak_associated_temperature_df['relative_datetime'] < xh_post), 'pre_post'] = 'post'

fever_peak_associated_temperature_df.loc[(fever_peak_associated_temperature_df['relative_datetime'] < 0) 
                              & (fever_peak_associated_temperature_df['relative_datetime'] > -1), 'pre_post'] = 'pre'

In [ ]:
# boxplot
sns.boxplot(x='pre_post', y='temperature', hue='pre_post', data=fever_peak_associated_temperature_df.reset_index(), showfliers=False)

In [ ]:
print(fever_peak_associated_temperature_df[fever_peak_associated_temperature_df['pre_post'] == 'pre']['temperature'].median(), f'(IQR: {fever_peak_associated_temperature_df[fever_peak_associated_temperature_df["pre_post"] == "pre"]["temperature"].quantile(0.25)} - {fever_peak_associated_temperature_df[fever_peak_associated_temperature_df["pre_post"] == "pre"]["temperature"].quantile(0.75)})')

print(fever_peak_associated_temperature_df[fever_peak_associated_temperature_df['pre_post'] == 'post']['temperature'].median(), f'(IQR: {fever_peak_associated_temperature_df[fever_peak_associated_temperature_df["pre_post"] == "post"]["temperature"].quantile(0.25)} - {fever_peak_associated_temperature_df[fever_peak_associated_temperature_df["pre_post"] == "post"]["temperature"].quantile(0.75)})')

In [ ]:
# mixed model
fever_peak_associated_temperature_df.dropna(subset=['pre_post'], inplace=True)
peak_fever_temperature_mixed_model = smf.mixedlm("temperature ~ pre_post", fever_peak_associated_temperature_df, groups=fever_peak_associated_temperature_df['pat_nr'])
mdf = peak_fever_temperature_mixed_model.fit()
# print formula
print(mdf.model.formula)
print(mdf.summary())

# compare pre / post cpp

In [ ]:
# compare pre / post cpp
associated_cpp_df['pre_post'] = 'pre'
associated_cpp_df.loc[associated_cpp_df['relative_datetime'] > 0, 'pre_post'] = 'post'
sns.boxplot(x='pre_post', y='cpp', hue='pre_post', data=associated_cpp_df, showfliers=False)

In [ ]:
associated_cpp_df[associated_cpp_df['pre_post'] == 'pre']['cpp'].median(), associated_cpp_df[associated_cpp_df['pre_post'] == 'post']['cpp'].median()

In [ ]:
# mixed model
associated_cpp_df.dropna(subset=['pre_post'], inplace=True)
peak_fever_cpp_mixed_model = smf.mixedlm("cpp ~ pre_post", associated_cpp_df, groups=associated_cpp_df['pat_nr'])
mdf = peak_fever_cpp_mixed_model.fit()
# print formula
print(mdf.model.formula)
print(mdf.summary())

In [ ]:
# compare 1h pre and Xh post, do not use data before -1h
xh_post = 12
fever_peak_associated_cpp_df = associated_cpp_df.copy()
fever_peak_associated_cpp_df['pre_post'] = np.nan
fever_peak_associated_cpp_df.loc[(fever_peak_associated_cpp_df['relative_datetime'] > 0, 'pre_post')] = 'post'
fever_peak_associated_cpp_df.loc[(fever_peak_associated_cpp_df['relative_datetime'] > 0)
                                & (fever_peak_associated_cpp_df['relative_datetime'] < xh_post), 'pre_post'] = 'post'
fever_peak_associated_cpp_df.loc[(fever_peak_associated_cpp_df['relative_datetime'] < 0)
                                & (fever_peak_associated_cpp_df['relative_datetime'] > -1), 'pre_post'] = 'pre'

In [ ]:
# boxplot
sns.boxplot(x='pre_post', y='cpp', hue='pre_post', data=fever_peak_associated_cpp_df.reset_index(), showfliers=False)

In [ ]:
print(fever_peak_associated_cpp_df[fever_peak_associated_cpp_df['pre_post'] == 'pre']['cpp'].median(), f'(IQR: {fever_peak_associated_cpp_df[fever_peak_associated_cpp_df["pre_post"] == "pre"]["cpp"].quantile(0.25)} - {fever_peak_associated_cpp_df[fever_peak_associated_cpp_df["pre_post"] == "pre"]["cpp"].quantile(0.75)})')

print(fever_peak_associated_cpp_df[fever_peak_associated_cpp_df['pre_post'] == 'post']['cpp'].median(), f'(IQR: {fever_peak_associated_cpp_df[fever_peak_associated_cpp_df["pre_post"] == "post"]["cpp"].quantile(0.25)} - {fever_peak_associated_cpp_df[fever_peak_associated_cpp_df["pre_post"] == "post"]["cpp"].quantile(0.75)})')

In [ ]:
# mixed model
fever_peak_associated_cpp_df.dropna(subset=['pre_post'], inplace=True)
peak_fever_cpp_mixed_model = smf.mixedlm("cpp ~ pre_post", fever_peak_associated_cpp_df, groups=fever_peak_associated_cpp_df['pat_nr'])
mdf = peak_fever_cpp_mixed_model.fit()
# print formula
print(mdf.model.formula)
print(mdf.summary())

# compare pre / post Heart rate

In [ ]:
# compare pre / post heart rate
associated_hr_df['pre_post'] = 'pre'
associated_hr_df.loc[associated_hr_df['relative_datetime'] > 0, 'pre_post'] = 'post'
sns.boxplot(x='pre_post', y='hr', hue='pre_post', data=associated_hr_df, showfliers=False)


In [ ]:
# compare 1h pre and Xh post, do not use data before -1h
xh_post = 12
fever_peak_associated_hr_df = associated_hr_df.copy()
fever_peak_associated_hr_df['pre_post'] = np.nan
fever_peak_associated_hr_df.loc[(fever_peak_associated_hr_df['relative_datetime'] > 0, 'pre_post')] = 'post'
fever_peak_associated_hr_df.loc[(fever_peak_associated_hr_df['relative_datetime'] > 0)
                                & (fever_peak_associated_hr_df['relative_datetime'] < xh_post), 'pre_post'] = 'post'
fever_peak_associated_hr_df.loc[(fever_peak_associated_hr_df['relative_datetime'] < 0)
                                & (fever_peak_associated_hr_df['relative_datetime'] > -1), 'pre_post'] = 'pre'


In [ ]:
# boxplot
sns.boxplot(x='pre_post', y='hr', hue='pre_post', data=fever_peak_associated_hr_df.reset_index(), showfliers=False)

In [ ]:
print(fever_peak_associated_hr_df[fever_peak_associated_hr_df['pre_post'] == 'pre']['hr'].median(), f'(IQR: {fever_peak_associated_hr_df[fever_peak_associated_hr_df["pre_post"] == "pre"]["hr"].quantile(0.25)} - {fever_peak_associated_hr_df[fever_peak_associated_hr_df["pre_post"] == "pre"]["hr"].quantile(0.75)})')
print(fever_peak_associated_hr_df[fever_peak_associated_hr_df['pre_post'] == 'post']['hr'].median(), f'(IQR: {fever_peak_associated_hr_df[fever_peak_associated_hr_df["pre_post"] == "post"]["hr"].quantile(0.25)} - {fever_peak_associated_hr_df[fever_peak_associated_hr_df["pre_post"] == "post"]["hr"].quantile(0.75)})')

In [ ]:
# mixed model
fever_peak_associated_hr_df.dropna(subset=['pre_post'], inplace=True)
peak_fever_hr_mixed_model = smf.mixedlm("hr ~ pre_post", fever_peak_associated_hr_df, groups=fever_peak_associated_hr_df['pat_nr'])
mdf = peak_fever_hr_mixed_model.fit()
# print formula
print(mdf.model.formula)
print(mdf.summary())

# compare pre / post ICP

In [ ]:
# compare pre / post icp
associated_icp_df['pre_post'] = 'pre'
associated_icp_df.loc[associated_icp_df['relative_datetime'] > 0, 'pre_post'] = 'post'
sns.boxplot(x='pre_post', y='icp', hue='pre_post', data=associated_icp_df, showfliers=False)
plt.show()

In [ ]:
# print pre / post ICP median (IQR)


## Time spent PtiO2 < 20 pre vs post

In [ ]:
ptio2_threshold = 20
adjust_for_monitoring_time = True

time_under_ptio2_threshold_df = pd.DataFrame()

for drug_admin in associated_ptio2_df.drug_start.unique():
    instance_associated_ptio2_df = associated_ptio2_df[associated_ptio2_df.drug_start == drug_admin]
    # Step 1: Identify the times when the values are below the threshold
    instance_associated_ptio2_df['below_threshold'] = instance_associated_ptio2_df['ptio2'] < ptio2_threshold
    
    # Step 2: Calculate the time difference between each row
    instance_associated_ptio2_df['time_difference'] = instance_associated_ptio2_df['datetime'].diff().dt.total_seconds()
    
    # Step 3: Mask the rows where the values are above the threshold, setting those time differences to 0
    instance_associated_ptio2_df['time_below_threshold'] = instance_associated_ptio2_df['time_difference'].where(instance_associated_ptio2_df['below_threshold'], 0)

    # Step 4: Sum up the time spent below the threshold
    time_below_threshold_pre_admin = instance_associated_ptio2_df[instance_associated_ptio2_df['relative_datetime'] < 0]['time_below_threshold'].sum()
    time_below_threshold_post_admin = instance_associated_ptio2_df[instance_associated_ptio2_df['relative_datetime'] > 0]['time_below_threshold'].sum()

    if adjust_for_monitoring_time:
        time_below_threshold_pre_admin = time_below_threshold_pre_admin / instance_associated_ptio2_df[instance_associated_ptio2_df['relative_datetime'] < 0]['time_difference'].sum()
        time_below_threshold_post_admin = time_below_threshold_post_admin / instance_associated_ptio2_df[instance_associated_ptio2_df['relative_datetime'] > 0]['time_difference'].sum()
    
    instance_time_under_ptio2_threshold_df = pd.DataFrame([instance_associated_ptio2_df.pat_nr.unique()[0], drug_admin, time_below_threshold_pre_admin, time_below_threshold_post_admin]).T
    instance_time_under_ptio2_threshold_df.columns=['pat_nr', 'drug_start', 'time_under_threshold_pre_s', 'time_under_threshold_post_s']
    time_under_ptio2_threshold_df = pd.concat([time_under_ptio2_threshold_df, instance_time_under_ptio2_threshold_df])


In [ ]:
time_under_ptio2_threshold_df = time_under_ptio2_threshold_df.melt(id_vars=['pat_nr', 'drug_start'], value_name='time_under_threshold')
time_under_ptio2_threshold_df = time_under_ptio2_threshold_df.rename(columns={'variable':'pre_post'})

if adjust_for_monitoring_time:
    time_under_ptio2_threshold_df.rename(columns={'time_under_threshold':'fraction_time_under_threshold'}, inplace=True)
    time_under_ptio2_threshold_df['time_under_threshold_h'] = time_under_ptio2_threshold_df['fraction_time_under_threshold'] * 12
    time_under_ptio2_threshold_df['time_under_threshold_s'] = time_under_ptio2_threshold_df['fraction_time_under_threshold'] * 12 * 3600
else:
    time_under_ptio2_threshold_df.rename(columns={'time_under_threshold':'time_under_threshold_s'}, inplace=True)
    time_under_ptio2_threshold_df['time_under_threshold_h'] = time_under_ptio2_threshold_df['time_under_threshold_s'] / 3600

time_under_ptio2_threshold_df.time_under_threshold_s = pd.to_numeric(time_under_ptio2_threshold_df.time_under_threshold_s) 
time_under_ptio2_threshold_df.loc[time_under_ptio2_threshold_df['pre_post'] == 'time_under_threshold_pre_s', 'pre_post'] = 'pre'
time_under_ptio2_threshold_df.loc[time_under_ptio2_threshold_df['pre_post'] == 'time_under_threshold_post_s', 'pre_post'] = 'post'


In [ ]:
time_under_ptio2_threshold_df

In [ ]:
time_under_threshold_mixed_model = smf.mixedlm("time_under_threshold_s ~ pre_post", time_under_ptio2_threshold_df, groups=time_under_ptio2_threshold_df['pat_nr'])
time_under_threshold_mdf = time_under_threshold_mixed_model.fit()

In [ ]:
# print formula
print(time_under_threshold_mdf.model.formula)
print(time_under_threshold_mdf.summary())

In [ ]:
time_under_threshold_mdf.pvalues[1]

In [ ]:
print(f'Pre: {time_under_ptio2_threshold_df[time_under_ptio2_threshold_df["pre_post"] == "pre"]["time_under_threshold_h"].median():.2f} h (IQR: {time_under_ptio2_threshold_df[time_under_ptio2_threshold_df["pre_post"] == "pre"]["time_under_threshold_h"].quantile(0.25):.2f}-{time_under_ptio2_threshold_df[time_under_ptio2_threshold_df["pre_post"] == "pre"]["time_under_threshold_h"].quantile(0.75):.2f})')
print(f'Post: {time_under_ptio2_threshold_df[time_under_ptio2_threshold_df["pre_post"] == "post"]["time_under_threshold_h"].median():.2f} h (IQR: {time_under_ptio2_threshold_df[time_under_ptio2_threshold_df["pre_post"] == "post"]["time_under_threshold_h"].quantile(0.25):.2f}-{time_under_ptio2_threshold_df[time_under_ptio2_threshold_df["pre_post"] == "post"]["time_under_threshold_h"].quantile(0.75):.2f})')


In [ ]:
fig = plt.figure(figsize=(5, 5))

ax = sns.boxplot(x='pre_post', y='time_under_threshold_h', data=time_under_ptio2_threshold_df, hue='pre_post', palette='Blues')

ax.set_ylabel(f'Time with PtO2 < {ptio2_threshold} mmHg (h)')
ax.set_xlabel('')

# add annotation of p-value (line between boxes)
# get the x and y coordinates
x1, x2 = 0, 1
y, h, col = time_under_ptio2_threshold_df['time_under_threshold_h'].max() + 0.4, 0.3, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h+0.1, f'p={time_under_threshold_mdf.pvalues[1]:.3f}', ha='center', va='bottom', color=col)


ax.set_ylim(top=14)

In [ ]:
# fig.savefig('/Users/jk1/Downloads/time_under_ptio2_threshold_pre_post.png', dpi=600, bbox_inches='tight')

## alternative comparator 
use time period with Temp < 37.5 as pre

In [ ]:
temperature_df.head()

In [ ]:
temperature_limit = 37.5
possibilities = [[-24, -12], [-36, -24], [-48, -36], [-60, -48], [-72, -60]]
missing_periods = []

pre_associated_ptio2_df = pd.DataFrame()
pre_associated_cpp_df = pd.DataFrame()
pre_associated_temperature_df = pd.DataFrame()
pre_associated_hr_df = pd.DataFrame()
pre_associated_lpr_df = pd.DataFrame()


for index, row in drug_administration_df.iterrows():
    # find a continuous time period with temperature < 37.5 before drug administration
    period_found = False
    for pos in possibilities:
        lower_bound = row['drug_start'] + pd.to_timedelta(pos[0], unit='h')
        upper_bound = row['drug_start'] + pd.to_timedelta(pos[1], unit='h')
        instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                            & (temperature_df['datetime'] >= lower_bound)
                                            & (temperature_df['datetime'] <= upper_bound)]
        if instance_associated_temperature_df['temperature'].max() < temperature_limit:
            period_found = True
            break
            
    if not period_found:
        missing_periods.append([row['pat_nr'], row['drug_start']])
        continue
    
    pre_instance_associated_ptio2_df = ptio2_df[(ptio2_df['pat_nr'] == row['pat_nr'])
                                            & (ptio2_df['datetime'] >= lower_bound) 
                                            & (ptio2_df['datetime'] <= upper_bound)]
    pre_instance_associated_ptio2_df['drug_start'] = row['drug_start']
    pre_instance_associated_ptio2_df['relative_datetime'] = (pre_instance_associated_ptio2_df['datetime'] - row['drug_start'] - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_ptio2_df = pd.concat([pre_associated_ptio2_df, pre_instance_associated_ptio2_df])

    pre_instance_associated_cpp_df = cpp_df[(cpp_df['pat_nr'] == row['pat_nr'])
                                        & (cpp_df['datetime'] >= lower_bound) 
                                        & (cpp_df['datetime'] <= upper_bound)]
    pre_instance_associated_cpp_df['drug_start'] = row['drug_start']
    pre_instance_associated_cpp_df['relative_datetime'] = (pre_instance_associated_cpp_df['datetime'] - row['drug_start'] - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_cpp_df = pd.concat([pre_associated_cpp_df, pre_instance_associated_cpp_df])
    
    pre_instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                        & (temperature_df['datetime'] >= lower_bound)
                                        & (temperature_df['datetime'] <= upper_bound)]
    pre_instance_associated_temperature_df['drug_start'] = row['drug_start']
    pre_instance_associated_temperature_df['relative_datetime'] = (pre_instance_associated_temperature_df['datetime'] - row['drug_start']
                                                                    - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_temperature_df = pd.concat([pre_associated_temperature_df, pre_instance_associated_temperature_df])
    
    pre_instance_associated_hr_df = hr_df[(hr_df['pat_nr'] == row['pat_nr'])
                                        & (hr_df['datetime'] >= lower_bound)   
                                        & (hr_df['datetime'] <= upper_bound)]
    pre_instance_associated_hr_df['drug_start'] = row['drug_start']
    pre_instance_associated_hr_df['relative_datetime'] = (pre_instance_associated_hr_df['datetime'] - row['drug_start']
                                                            - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_hr_df = pd.concat([pre_associated_hr_df, pre_instance_associated_hr_df])
    
    pre_instance_associated_lpr_df = lpr_df[(lpr_df['pat_nr'] == row['pat_nr'])
                                        & (lpr_df['datetime'] >= lower_bound)
                                        & (lpr_df['datetime'] <= upper_bound)]
    pre_instance_associated_lpr_df['drug_start'] = row['drug_start']
    pre_instance_associated_lpr_df['relative_datetime'] = (pre_instance_associated_lpr_df['datetime'] - row['drug_start']
                                                            - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_lpr_df = pd.concat([pre_associated_lpr_df, pre_instance_associated_lpr_df])
    
    
    
        

In [ ]:
allowed_ptio2_range = [0, 200]
allowed_cpp_range = [0, 200]
allowed_temperature_range = [30, 45]
allowed_hr_range = [0, 300]
allowed_lpr_range = [0, 100]

In [ ]:
# drop values outside of allowed range
pre_associated_ptio2_df = pre_associated_ptio2_df[(pre_associated_ptio2_df['ptio2'] >= allowed_ptio2_range[0]) & (pre_associated_ptio2_df['ptio2'] <= allowed_ptio2_range[1])]
pre_associated_cpp_df = pre_associated_cpp_df[(pre_associated_cpp_df['cpp'] >= allowed_cpp_range[0]) & (pre_associated_cpp_df['cpp'] <= allowed_cpp_range[1])]
pre_associated_temperature_df = pre_associated_temperature_df[(pre_associated_temperature_df['temperature'] >= allowed_temperature_range[0]) & (pre_associated_temperature_df['temperature'] <= allowed_temperature_range[1])]
pre_associated_hr_df = pre_associated_hr_df[(pre_associated_hr_df['hr'] >= allowed_hr_range[0]) & (pre_associated_hr_df['hr'] <= allowed_hr_range[1])]
pre_associated_lpr_df = pre_associated_lpr_df[(pre_associated_lpr_df['lpr'] >= allowed_lpr_range[0]) & (pre_associated_lpr_df['lpr'] <= allowed_lpr_range[1])]

In [ ]:
pre_associated_ptio2_df['rounded_relative_datetime'] = pre_associated_ptio2_df['relative_datetime'].round(1)
pre_associated_cpp_df['rounded_relative_datetime'] = pre_associated_cpp_df['relative_datetime'].round(1)
pre_associated_temperature_df['rounded_relative_datetime'] = pre_associated_temperature_df['relative_datetime'].round(1)
pre_associated_hr_df['rounded_relative_datetime'] = pre_associated_hr_df['relative_datetime'].round(1)
pre_associated_lpr_df['rounded_relative_datetime'] = pre_associated_lpr_df['relative_datetime'].round(1)

In [ ]:
len(missing_periods)

In [ ]:
post_associated_ptio2_df = associated_ptio2_df[associated_ptio2_df['relative_datetime'] > 0]
post_associated_cpp_df = associated_cpp_df[associated_cpp_df['relative_datetime'] > 0]
post_associated_temperature_df = associated_temperature_df[associated_temperature_df['relative_datetime'] > 0]
post_associated_hr_df = associated_hr_df[associated_hr_df['relative_datetime'] > 0]
post_associated_lpr_df = associated_lpr_df[associated_lpr_df['relative_datetime'] > 0]

In [ ]:
# plot pre and post in two subplots
plot_temperature = True
plot_hr = True
plot_lpr = False

fig, ax = plt.subplots(2, 1, figsize=(20, 20))
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=pre_associated_ptio2_df, ax=ax[0],
             label=f'PtiO2 (n={pre_associated_ptio2_df.pat_nr.nunique()})', legend=False)
# create second y axis for cpp values
ax2 = ax[0].twinx()
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=pre_associated_cpp_df, ax=ax2, color='magenta',
             label=f'CPP (n={pre_associated_cpp_df.pat_nr.nunique()})', legend=False)

if plot_temperature:
    ax3 = ax[0].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=pre_associated_temperature_df, ax=ax3, color='green',
                 label=f'Temperature (n={pre_associated_temperature_df.pat_nr.nunique()})', legend=False)
    ax3.set_ylabel('Temperature (°C)')
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))
    
if plot_hr:
    ax4 = ax[0].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=pre_associated_hr_df, ax=ax4, color='#7b002c',
                 label=f'HR (n={pre_associated_hr_df.pat_nr.nunique()})', legend=False)
    ax4.set_ylabel('HR (bpm)')
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120))
    
if plot_lpr:
    ax5 = ax[0].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=pre_associated_lpr_df.reset_index(), ax=ax5, color='orange',
                 label=f'LPR (n={pre_associated_lpr_df.pat_nr.nunique()})', legend=False)
    ax5.set_ylabel('LPR')
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))
    
ax[0].set_xlabel('Relative time from administration (h)')
ax[0].set_ylabel('PtiO2 (mmHg)')
ax2.set_ylabel('CPP (mmHg)')
ax2.set_ylim(60, 100)
ax2.grid(None)

# show legend
lines, labels = ax[0].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3
    
if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4
    
if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5
    
ax[0].legend(legend_lines, legend_labels, loc='upper right')

# post
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=post_associated_ptio2_df, ax=ax[1],
             label=f'PtiO2 (n={post_associated_ptio2_df.pat_nr.nunique()})', legend=False)
# create second y axis for cpp values
ax2 = ax[1].twinx()
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=post_associated_cpp_df, ax=ax2, color='magenta',
             label=f'CPP (n={post_associated_cpp_df.pat_nr.nunique()})', legend=False)


if plot_temperature:
    ax3 = ax[1].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=post_associated_temperature_df, ax=ax3, color='green',
                 label=f'Temperature (n={post_associated_temperature_df.pat_nr.nunique()})', legend=False)
    ax3.set_ylabel('Temperature (°C)')
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))
    
if plot_hr:
    ax4 = ax[1].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=post_associated_hr_df, ax=ax4, color='#7b002c',
                 label=f'HR (n={post_associated_hr_df.pat_nr.nunique()})', legend=False)
    ax4.set_ylabel('HR (bpm)')
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120))
    
if plot_lpr:
    ax5 = ax[1].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=post_associated_lpr_df.reset_index(), ax=ax5, color='orange',
                 label=f'LPR (n={post_associated_lpr_df.pat_nr.nunique()})', legend=False)
    ax5.set_ylabel('LPR')
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))
    
ax[1].set_xlabel('Relative time from administration (h)')
ax[1].set_ylabel('PtiO2 (mmHg)')
ax2.set_ylabel('CPP (mmHg)')
ax2.set_ylim(60, 100)
ax2.grid(None)

# show legend
lines, labels = ax[1].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3
    
if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4
    
if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5
    
ax[1].legend(legend_lines, legend_labels, loc='upper right')

ax[0].set_title('Pre administration (afebrile)')

ax[1].set_title('Post administration')

In [ ]:
# fig.savefig('/Users/jk1/Downloads/pre_post_ptio2_cpp_temp_hr_lpr_plot.png', dpi=300, bbox_inches='tight')

In [ ]:
pre_associated_ptio2_df['pre_post'] = 'pre'
post_associated_ptio2_df['pre_post'] = 'post'
combined_pre_post_ptio2_df = pd.concat([pre_associated_ptio2_df, post_associated_ptio2_df])

In [ ]:
# boxplot pre / post ptio2
sns.boxplot(x='pre_post', y='ptio2', hue='pre_post', data=combined_pre_post_ptio2_df)

In [ ]:
pre_mixed_model = smf.mixedlm("ptio2 ~ pre_post", combined_pre_post_ptio2_df, groups=combined_pre_post_ptio2_df['pat_nr'])
pre_mdf = pre_mixed_model.fit()
print(pre_mdf.summary())

plot every single patient

In [ ]:
for administration_row in drug_administration_df.iterrows():
    
    patient_ptio2_df = associated_ptio2_df[(associated_ptio2_df['pat_nr'] == administration_row[1]['pat_nr'])
                                        & (associated_ptio2_df['drug_start'] == administration_row[1]['drug_start'])]
    patient_cpp_df = associated_cpp_df[(associated_cpp_df['pat_nr'] == administration_row[1]['pat_nr'])
                                        & (associated_cpp_df['drug_start'] == administration_row[1]['drug_start'])]
    if patient_ptio2_df.shape[0] == 0 or patient_cpp_df.shape[0] == 0:
        print(f'No data for patient {administration_row[1]["pat_nr"]}, at {administration_row[1]["drug_start"]}')
        continue
        
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=patient_ptio2_df, ax=ax, label='PtiO2', legend=False)
    # create second y axis for cpp values
    ax2 = ax.twinx()
    sns.lineplot(x='rounded_relative_datetime', y='cpp', data=patient_cpp_df, ax=ax2, color='magenta', label='CPP', legend=False)
    
    ax.set_xlabel('Relative time from administration (h)')
    ax.set_ylabel('PtiO2 (mmHg)')
    ax2.set_ylabel('CPP (mmHg)')
    
    # ax2.set_ylim(60,100)
    ax2.grid(None)
    
    # show legend
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines + lines2, labels + labels2, loc='upper right')
    
    # # add vertical line at x=0 with text label with "Diclofenac administration"
    # ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
    # ax.text(0.5, 0.12, 'Diclofenac 75mg', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    # 
    plt.title(f'Patient {administration_row[1]["pat_nr"]} at {administration_row[1]["drug_start"]}')
    plt.show()
    